In [3]:
# Install packages
%pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\38sha\\Lib\\site-packages\\langchain\\document_loaders\\json_loader.py'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter 1.0.0 requires notebook, which is not installed.


In [ ]:
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

# Splitting documents

In [ ]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('../data sources/wikipedia_naruto.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [ ]:
documents_1[:300]

In [ ]:
# Document Splitting
chunk_size = 200
chunk_overlap = 10

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split_1 = splitter.split_text(documents_1)
split_1 = splitter.create_documents(split_1)

In [ ]:
# Load txt documents
reader = TextLoader('../data sources/wikipedia_snake.txt')
reader = reader.load()
print(len(reader))
documents_2 = reader[0]

In [ ]:
documents_2.page_content[:300]

In [ ]:
# Document Splitting
split_2 = splitter.split_text(documents_2.page_content)
split_2 = splitter.create_documents(split_2)

# Embedding and storing

In [ ]:
# Load embeddings instructor
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-xl', model_kwargs={'device':'cuda'}
)

# Implement embeddings
db = FAISS.from_documents(split_1, instructor_embeddings)

# Save db
db.save_local('vector store/naruto')

In [ ]:
# Implement embeddings for the second doc
db_2 = FAISS.from_documents(split_2, instructor_embeddings)

# Save db
db_2.save_local('vector store/snake')

In [ ]:
# Merge two DBs
db.merge_from(db_2)
db.save_local('vector store/naruto_snake')

# Answer retrieval and generation

In [ ]:
# Load the Hugging Face token
from google.colab import userdata
token = userdata.get('huggingface_write')

In [ ]:
# Load db
loaded_db = FAISS.load_local(
    'vector store/naruto_snake', instructor_embeddings, allow_dangerous_deserialization=True
)

In [ ]:
# Retrieve answer
question = 'what is naruto?'

search = loaded_db.similarity_search(question)
search

In [ ]:
# Query more or less text chunks
search = loaded_db.similarity_search(question, k=6)
search

In [ ]:
search_scores = loaded_db.similarity_search_with_score(question)
search_scores

In [ ]:
loaded_db.similarity_search_with_score(question, k=8)

In [ ]:
temperature = 1
max_length = 300
llm_model = 'tiiuae/falcon-7b-instruct'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [ ]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
response

In [ ]:
# Ask a question
question = 'what is naruto?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'how to cook rice?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'hi, how are you?'
response = qa({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

## Try higher temperature

In [ ]:
temperature = 3
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [ ]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It is hallucinating. Need to lower the temperature

In [ ]:
temperature = 1.5
max_length = 600
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_high_temp = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [ ]:
# Ask a question
question = 'what is naruto?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'what happened with the seven tailed beasts?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It answered well in the beginning and then hallucinated.

In [ ]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation
# It also answered well in the beginning and then hallucinated.

In [ ]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'how to cook rice?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'hi, how are you?'
response = qa_high_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

## Try lower temp

In [ ]:
temperature = 0.8
max_length = 300
llm_model = 'bigscience/bloom'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

# Create the chatbot
qa_low_temp = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
)

In [ ]:
# Ask a question
question = 'what is naruto?'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'what happened with the 7 tailed beasts?'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'summarize about venomous snakes'
response = qa_low_temp({'query': question})
answer = response.get('result').split('Helpful Answer:')[1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

## Try LangChain memory

In [ ]:
temperature = 1
max_length = 400
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load LLM
llm = HuggingFaceHub(
    repo_id=llm_model,
    model_kwargs={'temperature': temperature, 'max_length': max_length},
    huggingfacehub_api_token=token
)

memory = ConversationBufferWindowMemory(
    k=2,
    memory_key="chat_history",
    output_key="answer",
    return_messages=True,
)

qa_conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type='stuff',
    retriever=loaded_db.as_retriever(),
    return_source_documents=True,
    memory=memory,
)

In [ ]:
question = 'who is naruto?'
response = qa_conversation({'question': question})
response

In [ ]:
# Ask a question
# question = 'who is naruto?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'do you know whom I am talking about?'
response = qa_conversation({'question': question})
response

In [ ]:
# Ask a question
# question = 'do you know whom I am talking about?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'who is his team member?'
response = qa_conversation({'question': question})
response

In [ ]:
response.get('answer').split('Standalone question:')[2]

In [ ]:
# Ask a question
# question = 'who is his team member?'
# response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'who are the other teams?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
memory.load_memory_variables({})

In [ ]:
# Ask a question
question = 'are there any species similar to snakes?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'are they the only limbless animals?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation

In [ ]:
# Ask a question
question = 'Hi, i am a data scientist. How are you?'
response = qa_conversation({'question': question})
answer = response.get('answer').split('Helpful Answer:')[-1].strip()
explanation = response.get('source_documents', [])
print(answer)
explanation